In [ ]:
import pymongo
from pymongo import MongoClient
from bson import ObjectId
import pandas as pd
from pandas import DataFrame
from pandas import json_normalize
# from pandas.io.json import json_normalize
print ("pymongo version:", pymongo.version)

pd.set_option('display.max_columns', 1010)
pd.set_option('display.max_rows', 1010)

### Check status mongodb
* sudo service mongodb status
* sudo service mongodb start

In [ ]:
mongo_client = MongoClient('localhost', 27017)
host_info = mongo_client['HOST']
print ("\nhost:", host_info)

### List databases

In [ ]:
print(mongo_client.list_database_names())

In [ ]:
legalthings_db = mongo_client["legalthings"]

### Filter first features

In [ ]:
json_normalize(legalthings_db['processes_18'].find({})[0])

In [ ]:
df = json_normalize(legalthings_db['processes_18'].find({},{
    '_id': 1, 
    'name': 1, 
    'creation': 1, 
    'current.title': 1,
    'current.activation_date': 1, 
    'current.actor.email': 1, 
    'current.actor.name': 1,
    'private_data.incorporation_type': 1
}))

In [ ]:
cursor.count()

In [ ]:
cursor.count()

In [ ]:
cursor = legalthings_db['processes_18'].find({"private_data.meta_data.ah": {"$exists": True}})

In [ ]:
cursor.count()

In [ ]:
df = json_normalize(cursor, [['private_data', 'meta_data', 'ah']], ['_id'])

### Loop through all collections to gather features

In [ ]:
all_collections = legalthings_db.list_collection_names()

In [ ]:
all_feature_df = None

for collection_name in all_collections:
    if 'processes' in collection_name:
        df = json_normalize(
            legalthings_db[
                collection_name].find({},{
                '_id': 1,
                'name': 1,
                'creation': 1,
                'current.title': 1,
                'current.activation_date': 1,
                'current.actor.email': 1,
                'current.actor.name': 1,
                'private_data.incorporation_type': 1
        }))
        
        cursor = legalthings_db[
            collection_name].find({
            "private_data.meta_data.ah": {"$exists": True}
        })
        
        naw_df = json_normalize(cursor, [['private_data', 'meta_data', 'ah']], ['_id'])
        
        features_df = pd.merge(df, naw_df, on='_id', how='left')
        
        if all_feature_df is None:
            all_feature_df = features_df
        else:
            all_feature_df = pd.concat([all_feature_df, features_df])
        
        print(collection_name)

In [ ]:
all_feature_df.shape, len(all_feature_df['_id'].unique())

In [ ]:
all_feature_df.groupby('current.title')['_id'].nunique().sort_values(ascending=False)

In [ ]:
all_feature_df.head()